In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.InteractiveSession()
v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
tf.clip_by_value(v, 2.5, 4.5).eval() # 截断

array([[2.5, 2.5, 3. ],
       [4. , 4.5, 4.5]], dtype=float32)

In [3]:
v1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
v2 = tf.constant([[4.0, 6.0], [7.0, 8.0]])
print((v1*v2).eval()) # 矩阵逐元素相乘
print(tf.matmul(v1, v2).eval()) # 矩阵乘法

[[ 4. 12.]
 [21. 32.]]
[[18. 22.]
 [40. 50.]]


In [4]:
v = tf.constant([[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]])
tf.reduce_mean(v).eval()

3.5

In [7]:
# 自定义损失函数
v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])
print(tf.greater(v1, v2).eval()) # 逐元素比较
print(tf.where(tf.greater(v1, v2), v1, v2).eval()) # 如果第一个参数为True，则输出第二个参数，否则输出第三个参数(逐元素进行)

[False False  True  True]
[4. 3. 3. 4.]


In [1]:
# l1/l2 regularizer
import tensorflow as tf
weights = tf.constant([[1.0, -2.0], [-3.0, 4.0]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(weights))) # l1正则化， 0.5为正则化比例
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weights)))

5.0
7.5


In [7]:
# 通过集合正则化多组不同的w
# 在初始化参数时就将其加入正则化参数集合
def get_weight(shape, lam):
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lam)(var))
    return var
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
batch_size = 8
layer_dimension = [2, 10, 10, 10, 1]
n_layers = len(layer_dimension)
current_layer = x # 当前层
in_dimension = layer_dimension[0] # 输入层节点数
for i in range(1, n_layers):
    out_dimension = layer_dimension[i] # 当前层输出维度
    weight = get_weight([in_dimension, out_dimension], 0.001) # 权重
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))
    current_layer = tf.nn.relu(tf.matmul(current_layer, weight) + bias) # 当前层输出， 下层输入
    in_dimension = layer_dimension[i] # 下层输入维度
mse_loss = tf.reduce_mean(tf.square(y_-current_layer))
tf.add_to_collection('losses', mse_loss) # 损失集合中已有正则化损失
loss = tf.add_n(tf.get_collection('losses')) # get_collection获取集合所有元素，相加得到最终损失

In [8]:
loss

<tf.Tensor 'AddN:0' shape=() dtype=float32>

In [9]:
# 滑动平均模型
# 用于控制参数的更新速度, 在旧参数与新参数间滑动，得到滑动平均值（影子变量）
# shadow_variable = decay * shadow_variable + (1-decay)*variable
import tensorflow as tf
# 定义一个变量用于计算滑动平均， 变量的初值为0
v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False) # step为训练轮次，用于控制decay的大小 decay = min(decay, (1+step)/(10+step))
# 定义一个滑动平均类， 初始化时给定衰减率0.99及控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)
# 定义一个更新变量滑动平均的操作，每次执行该操作时，列表中的变量都会被更新
maintain_averages_op = ema.apply([v1])
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # 通过ema.average(v1)获取滑动平均之后影子变量的取值。
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新变量v1的值到5
    sess.run(tf.assign(v1, 5))
    # 更新滑动平均值
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新step的值为10000
    sess.run(tf.assign(step, 10000))
    # 更新v1的值为10
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 再次更新滑动平均值
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
